In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import re
import pandas as pd
import csv

input_folder_path = '/content/drive/MyDrive/data collection'  #place with the path to the folder containing your .txt files


folder_path = '/content/drive/MyDrive/data_collection_op_table/'
output_folder_path1 = '/content/drive/MyDrive/final_mean_data_ascending/'
output_folder_path2 = '/content/drive/MyDrive/final_mean_data_descending/'
output_file_all_values_ascending = '/content/drive/MyDrive/all_ascending_values'
output_file_all_values_descending = '/content/drive/MyDrive/all_descending_values'
output_file_final_mean = '/content/drive/MyDrive/Final_Calc_Mean'
output_folder_path_table = '/content/drive/MyDrive/data_collection_op_table/'










Mounted at /content/drive


In [ ]:
def readon(file_path):
  with open(file_path, 'r') as file:
      lines = file.readlines()

# Remove leading and trailing whitespaces from each line
  lines = [line.strip() for line in lines]

# Define a regular expression pattern to split the lines based on spaces
  pattern = r'\s+'

# Split each line using the regex pattern and create a list of lists
  data = [re.split(pattern, line) for line in lines]

  num_columns = max(len(row) for row in data)

  cols = ['Pressure', 'Temperature', 'Conductivity', 'Oxygen', 'Fluorescence', 'Turbidity', 'Depth', 'Density', 'Salinity', 'Sound_vel', 'Flag']

  if num_columns > len(cols):
        cols.extend(['Extra_{}'.format(i) for i in range(len(cols), num_columns)])
  df=pd.DataFrame(data,columns=cols)
  df.drop('Flag', axis = 1, inplace = True)

  df.drop(0, axis = 0, inplace = True)  #Deletes the row which has 'None' values

  df = df.astype(float)   #Converts the values from string format to float format
  df.reset_index(drop=True, inplace=True)
  return df

In [ ]:
#Code to convert .txt files to table
def read_table_from_file(path):
    table_data = []
    reading_table = False
#************************************************************
    with open(path, 'r') as file:
        for line in file:
            line = line.strip()
            if 'System UpLoad Time' in line:
              system_upload_time_line = line
            if 'NMEA Latitude' in line:
              nmea_latitude_line = line
            if 'NMEA Longitude' in line:
              nmea_longitude_line = line
            # Skip lines until the table starts
            if not reading_table:
                if line == '*END*':
                    reading_table = True
                continue

            # Split line by one or more spaces
            columns = line.split()

            # Ignore empty lines
            if not columns:
                continue

            # Append the table row to the data list
            table_data.append(columns)
    lines_to_add = [system_upload_time_line, nmea_latitude_line, nmea_longitude_line]
    return table_data, lines_to_add
#*********************************************************


#Function iteration
def iteration(df, max_depth_index, output_file1, output_file2, output_file_path3, output_file_path4, output_file_path5, lines_to_add):
    # Slice the DataFrame to include the second half
    first_half = df.iloc[:max_depth_index+1]
    second_half = df.iloc[max_depth_index:]
    first_half = first_half[first_half['Depth']<=5]
    first_half.reset_index(drop = True)
    first_half.to_csv(output_file_path3, header = True)
    second_half = second_half[second_half['Depth']<=5]
    second_half.reset_index(drop = True)
    second_half.to_csv(output_file_path4, header = True)

    # Calculate the means for all column
    means1 = first_half.mean()
    means2 = second_half.mean()
    means1 = pd.DataFrame(means1)
    means2 = pd.DataFrame(means2)
    means1 = means1.transpose()
    means2 = means2.transpose()
    mean_add = means1 + means2
    mean_add = pd.DataFrame(mean_add)
    mean_final = mean_add/2
    mean_final = pd.DataFrame(mean_final)
    mean_final.to_csv(output_file_path5, header = True)

    # Save means to a separate output file
    means1.to_csv(output_file1, header=True)
    means2.to_csv(output_file2, header = True)
    print("The 1st dataframe is", means1)
    print("The 2nd dataframe is", means2)
    print("The addition of the means is:", mean_add)
    print("The mean of the means1 and means2 is:", mean_final)
    with open(output_file_path5, "r") as f1:
        old_content = f1.read()

    with open(output_file_path5, "w", newline="") as f1:
        writer = csv.writer(f1)
        for lat in lines_to_add:
          writer.writerow([lat])
        f1.write(old_content)

#******************************************************************************






In [ ]:
# Iterate over files in the input folder
for filename in os.listdir(input_folder_path):
    input_file_path = os.path.join(input_folder_path, filename)

    # Check if the item is a file and has .txt extension
    if os.path.isfile(input_file_path) and filename.endswith('.txt'):

        #Read Time, Lat, Lon and Time_
        system_upload_time_line = None
        nmea_latitude_line =None
        nmea_longitude_line = None
        # Read the table data from the input file
        table = read_table_from_file(input_file_path)
        lines_to_add = table[1]
        table = table[0]

        # Create the output file path
        output_file_path_table = os.path.join(output_folder_path_table, filename)

        with open(output_file_path_table, 'w') as output_file:
            for row in table:
              output_file.write('\t'.join(row) + '\n')

        print(f'Table data saved to file: {output_file_path_table}')
        file_path = output_file_path_table

        #Reading of the output table begins from here
        df = readon(file_path)
        max_depth_index = df['Depth'].idxmax()

        # Generate output file name based on input file name
        output_file_name1 = f"{os.path.splitext(filename)[0]}_output.csv"
        output_file_name2 = f"{os.path.splitext(filename)[0]}_output.csv"
        output_file_name3 = f"{os.path.splitext(filename)[0]}.csv"
        output_file_name4 = f"{os.path.splitext(filename)[0]}.csv"
        output_file_name5 = f"{os.path.splitext(filename)[0]}.csv"


        output_file_path1 = os.path.join(output_folder_path1, output_file_name1)
        output_file_path2 = os.path.join(output_folder_path2, output_file_name2)
        output_file_path3 = os.path.join(output_file_all_values_ascending, output_file_name3)
        output_file_path4 = os.path.join(output_file_all_values_descending, output_file_name4)
        output_file_path5 = os.path.join(output_file_final_mean, output_file_name5)


        # Call the iteration function and pass the DataFrame and output file path
        iteration(df, max_depth_index, output_file_path1, output_file_path2, output_file_path3, output_file_path4, output_file_path5, lines_to_add)

Table data saved to file: /content/drive/MyDrive/data_collection_op_table/SSD_079_NCTD_057.txt
The 1st dataframe is    Pressure  Temperature  Conductivity    Oxygen  Fluorescence  Turbidity  \
0      3.75    25.217825      5.364219  4.334675      0.057725   0.941388   

   Depth   Density   Salinity  Sound_vel  
0  3.726  23.46605  35.252788   1535.265  
The 2nd dataframe is    Pressure  Temperature  Conductivity    Oxygen  Fluorescence  Turbidity  \
0      2.55    25.225325      5.276297  3.003469      0.045269   0.946113   

    Depth    Density   Salinity    Sound_vel  
0  2.5335  23.002312  34.638881  1534.605625  
The addition of the means is:    Pressure  Temperature  Conductivity    Oxygen  Fluorescence  Turbidity  \
0       6.3     50.44315     10.640516  7.338144      0.102994     1.8875   

    Depth    Density   Salinity    Sound_vel  
0  6.2595  46.468362  69.891669  3069.870625  
The mean of the means1 and means2 is:    Pressure  Temperature  Conductivity    Oxygen  Fluore